<a href="https://colab.research.google.com/github/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/Dev/Logic/UserManager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
%pip install -q importnb

try:
    # Clone the GitHub repository if not already present
    if not os.path.exists("/content/Project-Pheonix"):
        !git clone https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix.git /content/Project-Pheonix

    # Change directory to project root
    %cd /content/Project-Pheonix

    # Checkout the 'dev' branch
    !git fetch origin -q
    !git checkout Dev -q

    # Add project directory to Python path
    sys.path.append("/content/Project-Pheonix/Logic")
    from importnb import Notebook
    with Notebook():
        import CloudDB as dbService


except Exception as e:
    print("❌ Setup failed:", str(e))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.7 MB/s eta 0:00:00
Cloning into '/content/Project-Pheonix'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 527 (delta 62), reused 13 (delta 13), pack-reused 425 (from 1)
Receiving objects: 100% (527/527), 1.90 MiB | 13.30 MiB/s, done.
Resolving deltas: 100% (289/289), done.
/content/Project-Pheonix


In [ ]:
def login(username, password):
    if not username or not password:
        return "❌ Please fill in all fields.", None, False

    user_data = dbService.get_user_by_username(username)

    # Check if user exists
    if not user_data:
        return "❌ Invalid username or password.", None, False

    # Verify password
    if user_data["password"] != password:
        return "❌ Invalid username or password.", None, False

    # Return success message, username, and admin status
    if user_data.get("is_admin"):
        return f"✅ Welcome Admin {username}!", username, user_data.get("is_admin")
    else:
        return f"✅ Welcome {username}!", username, user_data.get("is_admin")


def register_account(username, password, confirm_password, is_admin):
    if not username or not password:
        return "❌ Please fill in all fields.", None, False
    if password != confirm_password:
        return "❌ Passwords do not match.", None, False

    # Use Firebase's direct method to check if user exists
    existing_user = dbService.get_user_by_username(username)

    if existing_user is not None:
        return "❌ Username already exists.", None, False

    user_data = {
        "password": password,
        "is_admin": is_admin,
        "coins": 2500
    }

    # Save user directly to Firebase
    result = dbService.insert_to_db_user(username, user_data)
    if result:
        return f"✅ User '{username}' registered successfully!", username, is_admin
    else:
        return "❌ Failed to register. Try again later.", None, False

def get_all_users():
    """Get all users from database except Admin"""
    try:
        users = dbService.get_from_db('users') or {}
        return users
    except Exception as e:
        print(f"Error getting users: {e}")
        return []

def update_user_coins(username, coins_to_add):
    """Update user's coins by adding the specified amount"""
    try:
        userExist = dbService.check_user_exists(username)
        if not userExist:
            return False, f"User '{username}' not found!"

        secsess = dbService.change_coins_for_user(username,coins_to_add)
        if secsess:
            return True, f"Added {coins_to_add} coins to {username}. New balance: {secsess} coins"
        else:
            return False, f"Error updating coins for {username}."

    except Exception as e:
        return False, f"Error updating coins: {str(e)}"